In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.datasets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=4, shuffle=False, num_workers=2)

170499072it [00:19, 8789485.90it/s]                               


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [20]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        num_features = 1
        size = x.size()[1:]
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
len(train_loader)

12500

In [8]:
for epoch in range(2):
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()

        if i % 2000 == 1999:
            print("Epoch: {}, Batch: {}, Loss: {}".format(epoch+1, i+1, running_loss/2000))
            running_loss = 0.0

Epoch: 1, Batch: 2000, Loss: 2.2385790459513664
Epoch: 1, Batch: 4000, Loss: 1.9300584586560725
Epoch: 1, Batch: 6000, Loss: 1.7061758731305599
Epoch: 1, Batch: 8000, Loss: 1.5926369809955359
Epoch: 1, Batch: 10000, Loss: 1.5365571353137493
Epoch: 1, Batch: 12000, Loss: 1.470420665755868
Epoch: 2, Batch: 2000, Loss: 1.4028293181061744
Epoch: 2, Batch: 4000, Loss: 1.381592670995742
Epoch: 2, Batch: 6000, Loss: 1.353023955911398
Epoch: 2, Batch: 8000, Loss: 1.3507136064395309
Epoch: 2, Batch: 10000, Loss: 1.313475228574127
Epoch: 2, Batch: 12000, Loss: 1.293453715853393


In [10]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)

        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(100 * correct / total)

55.36


In [14]:
len(test_loader), len(test_set)
# batch_size = 4

(2500, 10000)

In [21]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        c = (predicted == labels)

        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()     # C[i]=True 이면 1을 더함, False이면 0을 더함
            class_total[label] += 1
    
    print(class_correct, class_total)

for j in range(10):
    print("{}\t class accuracy: {}%".format(classes[j], 100 * class_correct[j] / class_total[j]))

[604.0, 817.0, 266.0, 430.0, 442.0, 451.0, 750.0, 624.0, 663.0, 489.0] [1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]
plane	 class accuracy: 60.4%
car	 class accuracy: 81.7%
bird	 class accuracy: 26.6%
cat	 class accuracy: 43.0%
deer	 class accuracy: 44.2%
dog	 class accuracy: 45.1%
frog	 class accuracy: 75.0%
horse	 class accuracy: 62.4%
ship	 class accuracy: 66.3%
truck	 class accuracy: 48.9%
